### DATA MAPPING


In [3]:
import pandas as pd
import numpy as np
import os

# Define file paths (update these with your actual file paths)
file_one_path = '../output/mapped_teus_goldenberg_June26_2025.xlsx'
file_two_path = '../proddata/teus_goldenberg_inventories.csv'

# Extract filenames for keyword detection
file_one_name = file_one_path.lower()
file_two_name = file_two_path.lower()

# Determine base_original based on keywords in filenames
if 'ugnayan' in file_one_name and 'ugnayan' in file_two_name:
    base_original = "https://bahay-ugnayan.rtdigit.net/inventory/view/"
elif 'kalayaan' in file_one_name and 'kalayaan' in file_two_name:
    base_original = "https://kalayaan.rtdigit.net/inventory/view/"
elif 'goldenberg' in file_one_name and 'goldenberg' in file_two_name:
    base_original = "https://teus-goldenberg.rtdigit.net/inventory/view/"
else:
    # Default case (use goldenberg if no keywords match)
    base_original = "https://teus-goldenberg.rtdigit.net/inventory/view/"

# Constant for qrcode_new
base_new = "https://inven.wpenginepowered.com/view-inventory-item/?accession_id="

# Load and process FILE TWO
df2 = pd.read_csv(file_two_path)
df2['_id_clean'] = df2['_id'].str.extract(r'ObjectId\((.*?)\)', expand=False)
df2['_id_clean'] = np.where(df2['_id_clean'].isna(), df2['_id'], df2['_id_clean'])

# Load FILE ONE
df1 = pd.read_excel(file_one_path, sheet_name='Sheet1')

# Create mapping from FILE TWO
mapping = df2.set_index('accessionId')['_id_clean'].to_dict()

# Map cleaned _id to FILE ONE and generate QR codes
df1['qrcode_original'] = df1['Accession ID Number'].map(mapping).apply(lambda x: base_original + x if pd.notna(x) else "")
df1['qrcode_new'] = base_new + df1['Accession ID Number'].astype(str)

# Save enhanced FILE ONE
# Ensure the output directory exists
output_dir = '../output/mapped_with_qr_code/'
os.makedirs(output_dir, exist_ok=True)

# Construct the output file path
output_filename = os.path.basename(file_one_path).replace('.xlsx', '_WITH_QR_CODES.xlsx')
output_path = os.path.join(output_dir, output_filename)

df1.to_excel(output_path, index=False)

print(f"Enhanced file saved to: {output_path}")
print(f"Used base URL: {base_original}")

Enhanced file saved to: ../output/mapped_with_qr_code/mapped_teus_goldenberg_June26_2025_WITH_QR_CODES.xlsx
Used base URL: https://teus-goldenberg.rtdigit.net/inventory/view/
